In [ ]:
import pandas as pd
import plotly.express as px  # (version 4.7.0)
import plotly.graph_objects as go

import dash  # (version 1.12.0) pip install dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [ ]:
external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css']
app=dash.Dash(__name__, external_stylesheets=external_stylesheets)
df = pd.read_csv("data.csv")
df.reset_index(inplace=True)


In [ ]:
import json
india_states = json.load(open("states_india.geojson", "r"))
state_id_map = {}
for feature in india_states["features"]:
    feature["id"] = feature["properties"]["state_code"]
    state_id_map[feature["properties"]["st_nm"]] = feature["id"]

#state_id_map
df["id"] = df["place"].apply(lambda x: state_id_map[x])
#df[:5]


In [ ]:
import plotly.io as pio
pio.renderers.default = 'browser'
fig = px.choropleth(
        df,
        locations='id',
        geojson=india_states,
        color='rainfall(in mm)',
        scope='asia',
        hover_name="place",
        hover_data=['rainfall(in mm)'],
        color_continuous_scale='matter'
    
    )
fig.update_geos(fitbounds="locations", visible=False)
fig.show()


In [ ]:
app.layout= html.Div([
        
    html.H1("Rainfal and Deforestation data State wise ", style={'text-align': 'center'}),

    dcc.Dropdown(id="slct_year",
                 options=[
                     {"label": "Deforestation", "value":'deforestation(in Ha)' },
                     {"label": "Rainfall", "value": 'rainfall(in mm)'},
                     ],
                 multi=False,
                 value='rainfall(in mm)',
                 style={'width': "40%"}
                 ),

    html.Div(id='output_container', children=[]),
    html.Br(),

    dcc.Graph(id='my_bee_map', figure={})
    ])

In [ ]:
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_bee_map', component_property='figure')],
    [Input(component_id='slct_year', component_property='value')]
)
def update_graph(option_slctd):
    print(option_slctd)
    print(type(option_slctd))

    container = "The year chosen by user was: {}".format(option_slctd)

    dff = df.copy()
    
    
    # Plotly Express
    fig = px.choropleth(
            dff,
            locations='id',
            geojson=india_states,
            color=str(option_slctd),
            scope='asia',
            hover_name="place",
            hover_data=['rainfall(in mm)'],
            color_continuous_scale='deep'
    )
    fig.update_geos(fitbounds="locations", visible=False)

    return container, fig

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)
